In [4]:
import os

import ifcopenshell
from ifcopenshell.util.element import get_psets

In [5]:
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\IFC_test\\notebooks'

In [41]:
import pandas as pd

df = pd.read_excel(
    "../src/02-01-01 _Конструктивные решения_ .xls",
    index_col=0,
    header=None,
    names=["GESN", "name"]
).dropna()
df.index = df.index.str[1:]
# df.index = df.index.str.upper()
df.name.to_dict()

{'06-01-001-16': 'Устройство фундаментных плит железобетонных плоских',
 '06-01-024-03': 'Устройство стен подвалов и подпорных стен железобетонных высотой до 3 м, толщиной до 300 мм',
 '06-01-041-01': 'Устройство перекрытий безбалочных толщиной до 200 мм на высоте от опорной площади до 6 м',
 '06-01-031-03': 'Устройство железобетонных стен и перегородок высотой до 3 м, толщиной 200 мм',
 '09-03-014-01': 'Монтаж связей и распорок из одиночных и парных уголков, гнутосварных профилей для пролетов до 24 м при высоте здания до 25 м',
 '09-03-015-01': 'Монтаж прогонов при шаге ферм до 12 м при высоте здания до 25 м',
 '09-03-012-01': 'Монтаж стропильных и подстропильных ферм на высоте до 25 м пролетом до 24 м массой до 3,0 т'}

In [36]:
df.index.str

In [7]:
import pickle
G = pickle.load(open('../src/new_AR.pickle', 'rb'))

In [7]:
model = ifcopenshell.open('../Amundsena_IFC_24/DOU_510_KJ0.ifc')

In [16]:
elements = list(model.by_type("IfcProduct"))
len(elements)

136

In [17]:
len(model.by_type("IfcBuildingStorey"))

2

In [14]:
{i.is_a() for i in model.by_type("IfcTypeObject")}

{'IfcColumnType',
 'IfcFootingType',
 'IfcSlabType',
 'IfcStairFlightType',
 'IfcStairType',
 'IfcWallType'}

In [12]:
from src.ifc_to_nx_converter import IfcToNxConverter
node_attributes = IfcToNxConverter.node_attributes

In [13]:
walls = model.by_type("IfcWall")
props = node_attributes(walls[0])
props

{'global_id': '212dsrGFn9lPwyKNEXCovJ',
 'name': 'Базовая стена:Монолиный жб - 200мм:418939',
 'is_a': 'IfcWall',
 'group_type': 'Базовая стена:Монолиный жб - 200мм',
 'ADCM_RD': 'КЖ1',
 'ADCM_GESN': '06-01-024-03',
 'ADCM_Level': 'L-01',
 'ADCM_Part': '3',
 'ADCM_Title': 'АИ-11 О/22',
 'ADCM_DIN': None,
 'ADCM_JobType': None,
 'Elevation': None,
 'coordinates': (62480.27619762918, -15.0, -100.0)}

In [19]:
walls[0].is_a("IfcProduct"), walls[0].is_a("IfcObject")

(True, True)

In [25]:
from neo4j import GraphDatabase

group_driver = GraphDatabase.driver(
    "bolt://localhost:7688",
    auth=("neo4j", "23109900")
)

In [28]:
print(group_driver.verify_connectivity())

None


In [30]:
q = """
CREATE (n:Element)
SET n = $props
"""
group_driver.execute_query(q, props=props)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x0000029542B292D0>, keys=[])

In [ ]:
{'ADCM_RD': 'AS',
 'ADCM_JobType': 'STP', #wbs3
 'ADCM_DIN': '342', # wbs3_id
 'ADCM_Level': 'L01', # wbs2
 'ADCM_Title': '01p', # wbs1
 'id': 849}

In [12]:
ifcopenshell.util.element.get_psets(element, psets_only=True).keys()

dict_keys(['Pset_WallCommon', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfWall', 'ADCM'])

In [13]:
ifcopenshell.util.element.get_psets(element).keys()

dict_keys(['Pset_WallCommon', 'Qto_WallBaseQuantities', 'Pset_EnvironmentalImpactIndicators', 'Pset_ReinforcementBarPitchOfWall', 'ADCM'])

In [62]:
placement = door.ObjectPlacement
placement.get_info()
relative_placement = placement.RelativePlacement
relative_placement.get_info()
location = relative_placement.Location
location

{'id': 1199227,
 'type': 'IfcLocalPlacement',
 'PlacementRelTo': #1199222=IfcLocalPlacement(#334001,#1199221),
 'RelativePlacement': #1199226=IfcAxis2Placement3D(#3,$,$)}

In [13]:
storeys = model.by_type("IfcBuildingStorey")
storeys[0].Elevation

-5000.0

In [17]:
get_psets(storeys[0], psets_only=True)

{'Pset_BuildingStoreyCommon': {'Reference': 'Уровень:С обозначением имени уровня',
  'AboveGround': 'UNKNOWN',
  'id': 350501}}

In [14]:
storeys[0].get_info()

{'id': 163,
 'type': 'IfcBuildingStorey',
 'GlobalId': '1oF2WbnqX8r9z94rbm$A8B',
 'OwnerHistory': #42=IfcOwnerHistory(#39,#5,$,.NOCHANGE.,$,$,$,1698128164),
 'Name': 'Техподполье',
 'Description': None,
 'ObjectType': 'Уровень:С обозначением имени уровня',
 'ObjectPlacement': #162=IfcLocalPlacement(#33,#161),
 'Representation': None,
 'LongName': 'Техподполье',
 'CompositionType': 'ELEMENT',
 'Elevation': -5000.0}

In [9]:
# doors = list(filter(lambda el: el.is_a("IfcDoor"), storey.ContainsElements[0].RelatedElements))
elements = list(storey.ContainsElements[0].RelatedElements)
len(elements)

225

In [10]:
types = set(el.is_a() for el in elements)
types

{'IfcBuildingElementProxy', 'IfcWall'}

In [34]:
get_coordinates(doors[0])

(27819.99984492928, 19775.004971843486, 0.0)

In [40]:
p_doors = list(filter(lambda x: get_coordinates(x) != (0.0, 0.0, 0.0), doors))
len(p_doors)

44

In [41]:
get_coordinates(p_doors[1])

(-1.0950316209346056e-09, 0.0, 0.0)

In [43]:
get_coordinates(p_doors[2])

(-250.0, 200.0, 0.0)

In [44]:
get_coordinates(p_doors[3])

(250.0, -200.0, 0.0)

In [4]:
classes = ("IfcWall",
    "IfcDoor",
    "IfcSlab",
    "IfcCovering",
    "IfcBuildingElementProxy",
    "IfcStair",
    "IfcRailing",
    "IfcSpace",
    # "IfcBuildingStorey",
    "IfcWindow",
    # "IfcBuilding",
    "IfcCurtainWall")
classes

('IfcWall',
 'IfcDoor',
 'IfcSlab',
 'IfcCovering',
 'IfcBuildingElementProxy',
 'IfcStair',
 'IfcRailing',
 'IfcSpace',
 'IfcWindow',
 'IfcCurtainWall')

In [9]:
for i in classes:
    el = model.by_type(i)[0]
    print(i, hasattr(el, "ObjectPlacement"))
    print(get_coordinates(el))
    print()

IfcWall True
(14129.799844930327, 16000.004971844253, -100.0)

IfcDoor True
(27819.99984492928, 19775.004971843486, 0.0)

IfcSlab True
(0.0, 0.0, 0.0)

IfcCovering True
(0.0, 0.0, 0.0)

IfcBuildingElementProxy True
(66445.95098959892, 13990.0, 0.0)

IfcStair True
(0.0, 0.0, 0.0)

IfcRailing True
(15474.40472776368, 3663.0047277640956, -130.0)

IfcSpace True
(0.0, 0.0, 0.0)

IfcWindow True
(67297.49999999975, 147.49999999999963, 500.0)

IfcCurtainWall True
(0.0, 0.0, 0.0)



In [15]:
slab = model.by_type("IfcSlab")[0]
slab.ObjectPlacement.RelativePlacement.get_info()

{'id': 392059,
 'type': 'IfcAxis2Placement3D',
 'Location': #3=IfcCartesianPoint((0.,0.,0.)),
 'Axis': None,
 'RefDirection': None}

In [17]:
bool(slab.ObjectPlacement.RelativePlacement.Axis)

False

In [23]:
doors

NameError: name 'doors' is not defined